In [1]:
### DO NOT CHANGE THIS CELL

%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

import clingo
from IPython.display import clear_output

In [2]:
def print_answer_sets(program):
    # Load the answer set program, and call the grounder
    control = clingo.Control()
    control.add("base", [], program)
    control.ground([("base", [])])
    # Define a function that will be called when an answer set is found
    # This function sorts the answer set alphabetically, and prints it
    def on_model(model):
        sorted_model = [str(atom) for atom in model.symbols(shown=True)]
        sorted_model.sort()
        print("Answer set: {{{}}}".format(", ".join(sorted_model)))
    # Ask clingo to find all models (using an upper bound of 0 gives all models)
    control.configuration.solve.models = 0
    # Call the clingo solver, passing on the function on_model for when an answer set is found
    answer = control.solve(on_model=on_model)
    # Print a message when no answer set was found
    if answer.satisfiable == False:
        print("No answer sets")

In [3]:
### NO NEED TO CHANGE THIS CELL
### (UNLESS YOU WANT TO SET A SOLVING TIMEOUT
###  OR USE A DIFFERENT NUMBER OF PARALLEL THREADS)
asp_program = r"""
#const t = 31.
possible_time(1..t).
{ time(T) : possible_time(T) }.
:- not time(1).
:- possible_time(T), possible_time(T+1), not time(T), time(T+1).
max_time(MaxTime) :- MaxTime = #max { T : time(T) }.

#const n = 9.
token(1..n-1).
location(1..n).
position(1, 2).
position(2, 3).
position(3, 4).
position(4, 5).
position(5, 6).
position(6, 7).
position(7, 8).
position(8, 9).

action(transfer(Token, Location)) :- token(Token), location(Location).
1 { do(T, A) : action(A) } 1 :- time(T), T < M, max_time(M).

% Fluents
fluent(position(Token, Location)) :- token(Token), location(Location).
state(1, position(Token, Location)) :- position(Token, Location).
make_true(T+1, position(Token, Location2)) :- do(T, transfer(Token, Location2)), state(T, position(Token, Location1)), Location1 != Location2.
make_false(T+1, position(Token, Location1)) :- do(T, transfer(Token, Location2)), state(T, position(Token, Location1)), Location1 != Location2.
state(T, Statement) :- time(T), fluent(Statement), state(T-1, Statement), not make_false(T, Statement).
state(T, Statement) :- time(T), fluent(Statement), make_true(T, Statement).

% No two tokens can be in the same location at the same time
:- time(T), token(Token1), token(Token2), location(Location),
    state(T, position(Token1, Location)), state(T, position(Token2, Location)), Token1 != Token2.
% No token can be at two locations at the same time
:- time(T), token(Token), location(Location1), location(Location2),
    state(T, position(Token, Location1)), state(T, position(Token, Location2)), Location1 != Location2.

% Define adjacent positions
adjacent(1, 2).
adjacent(1, 4).
adjacent(2, 3).
adjacent(2, 5).
adjacent(3, 6).
adjacent(4, 5).
adjacent(4, 7).
adjacent(5, 6).
adjacent(5, 8).
adjacent(6, 9).
adjacent(7, 8).
adjacent(8, 9).
adjacent(Location1, Location2) :- adjacent(Location2, Location1).

% You can only move to adjacent locations
:- time(T), do(T, transfer(Token, Location1)), state(T, position(Token, Location2)), not adjacent(Location1, Location2).

% goal
goal_reached(T) :- time(T),
    state(T, position(1, 9)),
    state(T, position(2, 8)),
    state(T, position(3, 7)),
    state(T, position(4, 6)),
    state(T, position(5, 5)),
    state(T, position(6, 4)),
    state(T, position(7, 3)),
    state(T, position(8, 2)).
:- max_time(M), not goal_reached(M).
% Once we reached the goal, stop the clock!
:- max_time(M), goal_reached(T), time(T), T < M.

% Find the shortest sequence
#minimize { 1, T : time(T) }.
"""

rest= """
"""

show = """
#show do/2.
"""

print_answer_sets(asp_program + show)

Answer set: {do(1,transfer(3,1)), do(10,transfer(5,8)), do(11,transfer(2,9)), do(12,transfer(1,6)), do(13,transfer(7,3)), do(14,transfer(8,2)), do(15,transfer(5,5)), do(16,transfer(6,8)), do(17,transfer(4,7)), do(18,transfer(3,4)), do(19,transfer(8,1)), do(2,transfer(4,4)), do(20,transfer(5,2)), do(21,transfer(6,5)), do(22,transfer(4,8)), do(23,transfer(3,7)), do(24,transfer(6,4)), do(25,transfer(4,5)), do(26,transfer(2,8)), do(27,transfer(1,9)), do(28,transfer(4,6)), do(29,transfer(5,5)), do(3,transfer(7,5)), do(30,transfer(8,2)), do(4,transfer(8,8)), do(5,transfer(5,9)), do(6,transfer(2,6)), do(7,transfer(1,3)), do(8,transfer(7,2)), do(9,transfer(8,5))}
